<a href="https://colab.research.google.com/github/juancpv/predictivemaintenance/blob/master/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Antes de empezar a entrenar nuestro sistema de predicción de fallos. Aclaremos que estaremos trabajando en una maquina virtual con sistema operativo Linux y que usaremos python como lenguaje de programación.
Cada vez que queramos usar un comando de terminal de ubuntu, añadiremos al inicio de la instruccion el simbolo de exclamación (!)
A continuación importamos todas las librerias que usaremos a lo largo de este tutorial. A medida que las usemos las explicaremos en mas detalle.



In [0]:
import keras  #Biblioteca de redes neuronales 
import pandas as pd  #Biblioteca  para manipulación de datos
import numpy as np #Biblioteca para soporte  de operaciones matematicas de alto nivel en matrices y vectores.
import matplotlib.pyplot as plt #biblioteca para manejo de graficos

# Some tools
from sklearn import preprocessing #Paquete que permite la estandarizacion de datoscomo beneficio para los algoritmos de aprendizaje.
from sklearn.metrics import confusion_matrix, recall_score, precision_score # Paquete para medir el performance del clasificador.
from keras.models import Sequential 
from keras.layers import Dense, Dropout, LSTM


  **DATASETS.**

Para este ejercicio nosotros haremos uso de un dataset de Azure. Los datos consisten en tres archivos de texto: training, testing y ground truth para el caso especifico de 100 motores de aeronaves. 


*   El dataset de entrenamiento consiste de un conjunto de series de tiempo multivariable (21 sensores) con la unidad de tiempo medida en ciclos.
*   El dataset de pruebas, tiene el mismo esquema que el de entrenamiento, con la unica diferencia que estos no incluyen la etiqueta de ocurrencia de falla.
*   El datset de ground truth contiene información de ciclos restantes de trabajo de los motores




A continuación se dan los comandos para obtener los datasets (recuerde que esto es una maquina con sistema operativo Linux).





In [0]:
!wget http://azuremlsamples.azureml.net/templatedata/PM_train.txt 
!wget http://azuremlsamples.azureml.net/templatedata/PM_test.txt
!wget http://azuremlsamples.azureml.net/templatedata/PM_truth.txt

Examinemos un poco los datos con los cuales vamos a trabajar (aqui comenzamos con el uso de PANDAS):

In [0]:
train_df = pd.read_csv('PM_train.txt', sep=" ", header=None)
train_df.head(10) # Por defecto el numero de filas sera 5.


Podemos observar como las dos ultimas columnas no tienen datos. Vamos a usar la funcion de pandas DROP para eliminar esas columnas.

In [0]:
train_df.drop(train_df.columns[[26,27]], axis=1, inplace=True)
train_df.head()


Dado que el dataset no contiene etiquetas para las  diferentes columnas, vamos a hacerlo tomando en cuenta el [template](https://gallery.azure.ai/Experiment/Predictive-Maintenance-Step-1-of-3-data-preparation-and-feature-engineering-2) de la pagina de Azure.

In [0]:
train_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']
train_df.head(10)

Hagamos lo mismo para los datos de prueba.

In [0]:
test_df = pd.read_csv('PM_test.txt', sep=" ", header=None)
test_df.drop(test_df.columns[[26, 27]], axis=1, inplace=True)
test_df.columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3',
                     's4', 's5', 's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14',
                     's15', 's16', 's17', 's18', 's19', 's20', 's21']
test_df.tail(5)

Miremos el archivo de ground truth:

In [0]:
truth_df = pd.read_csv('PM_truth.txt', sep= " ", header=None)
truth_df.head()

In [0]:
Escriba el codigo a continuación para quitar la columan sin información:

In [0]:
truth_df.drop(truth_df.columns[[1]], axis=1, inplace=True)
truth_df.columns = ['cycle']
truth_df.head()

**PRE-PROCESAMIENTO DE DATOS**
Antes de comenzar con el pre.rpocesamiento de datos. Vamos a comprender en que consisten los datos:
En el archivo de entrenamiento, cada motor empieza con un grado diferente de uso inicial y variaciones de manufactura. Se asume que cada motor opera normalmente al comienzo de cada serie de tiempo y comienza a degradarse en algun punto durante la serie del ciclo de operación. La degradación es progresiva y crece en magnitud. Cuando un nivel de umbral es alcanzado, el motor es considerado inseguro. Asi que el ultimo ciclo se puede considerar como el punto de falla del correspondiente motor.

Como una forma de practicar y ver lo que sucede en el dataset de entrenamiento. Visualice cual sería el punto de falla para cada motor: 


In [0]:
#organizar los datos por id y por ciclos:
train_df =train_df.sort_values(['id', 'cycle'])
#train_df.head()
ultimo_ciclo = train_df.groupby('id')['cycle'].max()
ultimo_ciclo

Como pudo notar existen 100 motores y por ejemplo el motor 1 fallará en el ciclo 192 y el motor 100 fallará en el ciclo 200.
Para el dataset de prueba, el esquema es identico al dataset de entrenamiento, solo que en este caso el ultimo ciclo no representa el punto de falla para un motor en particular.

El dataset ground truth tendrá entonces  el ciclo donde fallará cada motor del dataset de prueba.

Como un ejercicio muestre que para el motor 2 el ultimo ciclo en el dataset de prueba es el 49 y el ciclo en que sera considerado como falla será en el 98:

In [0]:
test_df = test_df.sort_values(['id' , 'cycle'])
ultimo_ciclo_test = test_df.groupby('id')['cycle'].max()
ultimo_ciclo_test.loc[2] , truth_df.loc[1]

Comencemos entonces generando etiquetas para los datos de entrenamiento. 

In [0]:
#Etiqueta: Generando una columana TVR (Tiempo de vida remanente)
tvr = pd.DataFrame(train_df.groupby('id')['cycle'].max()).reset_index()
tvr.columns = ['id', 'max']
train_df = train_df.merge(tvr, on=['id'], how='left')
train_df['TVR'] = train_df['max'] - train_df['cycle']
train_df.drop('max', axis=1, inplace=True)
train_df.head()

,id,cycle,setting1,setting2,setting3,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,TVR
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.3,47.47,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,191
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,47.49,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,190
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.3,47.27,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,189
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,47.13,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,188
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.3,47.28,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,187


Existen varias preguntas que se podrían intentar responder desde el punto de vista de mantenimiento predictivo. Dependiendo de la pregunta asi mismo sería el tipo de modelo de machine learning que deberiamos usar. Por ejemplo: podriamos usar un **modelo de regresión** si la pregunta fuese: ¿Cuantos ciclos de más tendrá un determinado motor antes de fallar?; podriamos usar un **clasificador multi clase** si la pregunta fuese: ¿Este motor fallará dentro del rango de ciclos [1, W0] o fallará dentro del rango de ciclos [wo+1, w1] o este no fallará dentro de w1 ciclos?
Nosotros nos centraremos en un **clasificador binario** que nos de respuesta a la siguiente pregunta: ¿Este motor fallará dentro de los w1 ciclos?
